In [1]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import opt_einsum as oe
import matplotlib.pyplot as plt
import os

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.models.dense.density_matrix import density_matrix_1, density_matrix_2
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense

from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.core.sparse.hamiltonian_big import hamiltonian_general_sparse
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense


# Check dencity matrix 

In [2]:
site = 3
state = 11
g = 1

In [3]:
HKV = hamiltonian_dense(state, site, g)

H = HKV[0]
V = HKV[2]

In [4]:
H_NO, _, _ = hamiltonian_general_dense(state, site, g)

In [5]:
eig_val, eig_vec = np.linalg.eigh(H)

In [6]:
eig_val_NO, eig_vec_NO = np.linalg.eigh(H_NO)

In [7]:
np.allclose(np.sort(eig_val), np.sort(eig_val_NO), atol=1e-20)

True

## Check 1-d, 2-d eigenvalue ocupation.

In [8]:
site = 3
state = 11
g = 1

In [9]:
H, K, V = hamiltonian_dense(state, site, g, periodic=False)

In [10]:
eig_val, eig_vec = np.linalg.eigh(H)

index = np.argmin(eig_val)
ground_state_vec = eig_vec[:, index]

In [11]:
D_1 = density_matrix_1(state, site, ground_state_vec, 0)
D_2 = density_matrix_2(state, site, ground_state_vec, 0, 1)

In [12]:
eig_val_D_1, eig_vec_D_1 = np.linalg.eigh(D_1)
eig_val_D_2, eig_vec_D_2 = np.linalg.eigh(D_2.reshape(state**2, state**2))

In [13]:
-np.sort(-eig_val_D_1)

array([7.54651646e-01, 2.25916655e-01, 1.80879355e-02, 7.31062572e-04, 6.08807310e-04, 2.31831557e-06, 1.57191401e-06, 2.01968122e-09, 1.85983175e-09, 1.20191882e-12, 9.36142721e-13])

In [14]:
-np.sort(-eig_val_D_2)

array([ 7.54651646e-01,  2.25916655e-01,  1.80879355e-02,  7.31062572e-04,  6.08807310e-04,  2.31831557e-06,  1.57191401e-06,  2.01968119e-09,  1.85983176e-09,  1.20190174e-12,  9.36153081e-13,  8.00813428e-17,  5.99123592e-17,  5.61264362e-17,  4.63001170e-17,  1.84541925e-17,  1.36114681e-17,  6.39980932e-18,  4.97363778e-18,  4.54169458e-18,  2.86840187e-18,  1.51733174e-18,  1.04984520e-18,  5.21153044e-19,  4.72975194e-19,  7.09514162e-20,  4.40145388e-20,  3.75966551e-20,  1.72731877e-20,  1.39447453e-20,  1.34295038e-20,  4.24623782e-21,  1.97554850e-21,  1.31592309e-21,  4.83012508e-22,  4.33977795e-22,  3.80308578e-22,  1.19898261e-22,  8.57054666e-23,  2.66814500e-23,  2.02810827e-23,  1.75043577e-23,  4.86519061e-24,  4.33712594e-24,  2.18509649e-24,  1.45843328e-24,  3.87967039e-25,  2.23178165e-25,  1.41832523e-25,  4.76147560e-26,  2.70592627e-26,  2.21738964e-26,  1.65413791e-26,  1.00783508e-26,  3.21177111e-27,  2.01565027e-27,  1.30913030e-27,  9.10904273e-28,  1.1644

In [15]:
index = np.argsort(-eig_val_D_1)

eig_vec_D_1[:, index[:2]]

array([[ 9.96827156e-01+0.j,  2.27816658e-17+0.j],
       [-4.16333634e-17+0.j,  7.07056968e-01+0.j],
       [-2.64154435e-19+0.j, -7.07056968e-01+0.j],
       [-5.62830149e-02+0.j, -3.60084301e-17+0.j],
       [-5.62830149e-02+0.j, -8.19843127e-17+0.j],
       [ 4.32288527e-19+0.j, -8.39309329e-03+0.j],
       [ 1.68838123e-20+0.j,  8.39309329e-03+0.j],
       [ 1.81731567e-04+0.j,  1.95334962e-17+0.j],
       [ 1.81731567e-04+0.j,  1.04587496e-16+0.j],
       [-5.79395138e-22+0.j,  1.12214274e-05+0.j],
       [-5.33487386e-23+0.j, -1.12214274e-05+0.j]])

## Plot graphs

In [16]:
def load_one_simulation_ED(sites: int, states: int, g: float, directory: str = ".") -> dict:
    """
    Loads eigenvalues, eigenvectors, and NO basis matrix for a specific (sites, states, g) system.

    Returns a dictionary with 'eigvals', 'eigvecs', and 'NO_basis'.
    """
    g_str = f"{g:.1f}"
    base = f"sites{sites}_states{states}_g{g_str}"

    eigvals_path = os.path.join(directory, f"eigvals_{base}.npy")
    eigvecs_path = os.path.join(directory, f"eigvecs_{base}.npy")

    try:
        eigvals = np.load(eigvals_path)
        eigvecs = np.load(eigvecs_path)

        return eigvals, eigvecs

    except FileNotFoundError as e:
        print(f"[Error] Missing file for (sites={sites}, states={states}, g={g_str}): {e}")
        return None

In [17]:
def load_one_simulation_NO(sites: int, states: int, g: float, directory: str = ".") -> dict:
    """
    Loads eigenvalues, eigenvectors, and NO basis matrix for a specific (sites, states, g) system.

    Returns a dictionary with 'eigvals', 'eigvecs', and 'NO_basis'.
    """
    g_str = f"{g:.1f}"
    base = f"sites{sites}_states{states}_g{g_str}"

    eigvals_path = os.path.join(directory, f"NO_eigvals_{base}.npy")
    eigvecs_path = os.path.join(directory, f"NO_eigvecs_{base}.npy")
    NO_basis_path = os.path.join(directory, f"NO_basis_{base}.npy")

    try:
        eigvals = np.load(eigvals_path)
        eigvecs = np.load(eigvecs_path)
        NO_basis = np.load(NO_basis_path)

        return eigvals, eigvecs, NO_basis

    except FileNotFoundError as e:
        print(f"[Error] Missing file for (sites={sites}, states={states}, g={g_str}): {e}")
        return {}

In [18]:
def scatter_and_fit(g, y, label, color, degree=10, visible_indices=None):
    """
    Plot selected points from (g, y), fit to full data, and draw the fit curve.

    Parameters:
        g, y: full data arrays
        label: label for legend
        color: color for plot
        degree: degree of polynomial fit
        visible_indices: indices of points to show in the scatter plot
    """
    g = np.asarray(g)
    y = np.asarray(y)

    if visible_indices is not None:
        g_visible = g[visible_indices]
        y_visible = y[visible_indices]
    else:
        g_visible = g
        y_visible = y

    # Show only selected data points
    plt.scatter(g_visible, y_visible, label=f"{label}", color=color, s=30)

    # Fit to full data
    coeffs = np.polyfit(g, y, degree)
    poly_fn = np.poly1d(coeffs)

    # Plot fitted polynomial
    g_smooth = np.linspace(g.min(), g.max(), 300)
    y_smooth = poly_fn(g_smooth)
    plt.plot(g_smooth, y_smooth, linestyle='--', color=color)

In [19]:
file_path_ED = "./quant_rotor/data/ED_analisis_data"
file_path_ED_s = "./quant_rotor/dataED_analisis_data_sparce"
file_path_T_amplitudes = "./quant_rotor/dataT_amplitudes_energy"

In [20]:
si3_st11_ED = []
si3_st5_NO = []
si5_st5_NO = []
si7_st3_NO = []
g_array = []

for g_index in range(1, 51):
    g = g_index / 10
    g_array.append(g)

    eig_val, eig_vec = load_one_simulation_ED(3, 11, g, file_path_ED)

    index_array = np.argmin(eig_val)
    ground_state_vec = eig_vec[:, index]

    D_1 = density_matrix_1(3, 11, ground_state_vec, 0)
    D_2 = density_matrix_2(3, 11, ground_state_vec, 0, 1)

    eig_val_D_1, eig_vec_D_1 = np.linalg.eigh(D_1)
    eig_val_D_2, eig_vec_D_2 = np.linalg.eigh(D_2.reshape(state**2, state**2))

    si3_st11_ED.append(-np.sort(-eig_val_D_1)[:2], -np.sort(-eig_val_D_2)[:2])

    eig_val, eig_vec, _ = load_one_simulation_NO(3, 5, g, file_path_ED)

    index_array = np.argmin(eig_val)
    ground_state_vec = eig_vec[:, index]

    D_1 = density_matrix_1(3, 11, ground_state_vec, 0)
    D_2 = density_matrix_2(3, 11, ground_state_vec, 0, 1)

    eig_val_D_1, eig_vec_D_1 = np.linalg.eigh(D_1)
    eig_val_D_2, eig_vec_D_2 = np.linalg.eigh(D_2.reshape(state**2, state**2))

    si3_st5_NO.append(-np.sort(-eig_val_D_1)[:2], -np.sort(-eig_val_D_2)[:2])

    eig_val, eig_vec, _ = load_one_simulation_NO(5, 5, g, file_path_ED)

    index_array = np.argmin(eig_val)
    ground_state_vec = eig_vec[:, index]

    D_1 = density_matrix_1(3, 11, ground_state_vec, 0)
    D_2 = density_matrix_2(3, 11, ground_state_vec, 0, 1)

    eig_val_D_1, eig_vec_D_1 = np.linalg.eigh(D_1)
    eig_val_D_2, eig_vec_D_2 = np.linalg.eigh(D_2.reshape(state**2, state**2))

    si5_st5_NO.append(-np.sort(-eig_val_D_1)[:2], -np.sort(-eig_val_D_2)[:2])

    eig_val, eig_vec, _ = load_one_simulation_NO(7, 3, g, file_path_ED)

    index_array = np.argmin(eig_val)
    ground_state_vec = eig_vec[:, index]

    D_1 = density_matrix_1(3, 11, ground_state_vec, 0)
    D_2 = density_matrix_2(3, 11, ground_state_vec, 0, 1)

    eig_val_D_1, eig_vec_D_1 = np.linalg.eigh(D_1)
    eig_val_D_2, eig_vec_D_2 = np.linalg.eigh(D_2.reshape(state**2, state**2))

    si3_st11_ED.append(-np.sort(-eig_val_D_1)[:2], -np.sort(-eig_val_D_2)[:2])

IndexError: index 1331 is out of bounds for axis 0 with size 1331

In [ ]:
si3_st11_ED = np.array(si3_st11_ED)
si3_st5_NO = np.array(si3_st5_NO)
si5_st5_NO = np.array(si5_st5_NO)
si7_st3_NO = np.array(si7_st3_NO)
g_array = np.array(g_array)

dim_fit = 20

visible_idx = np.r_[0:10, 19, 49]

plt.figure(figsize=(20, 6))

for y_data, label, color in zip(
    [si3_st11_ED, si3_st5_NO, si5_st5_NO, si7_st3_NO],
    ['Site 3 State 11 ED', 'Site 3 State 5 NO', 'Site 5 State 5 NO', 'Site 7 State 3 NO'],
    ['blue', 'orange', 'green', 'red']
):

    # Fit and plot
    scatter_and_fit(g_array, y_data, label, color, degree=dim_fit)

# Axis formatting
plt.xticks(g_array, rotation=45)
plt.xlabel("g-values")
plt.ylabel("Energy Gap")
plt.title("Energy Gap Variation With g")
plt.legend()
plt.grid(True)
plt.show()

si3_st11_ED = np.array(si3_st11_ED)
si3_st5_NO = np.array(si3_st5_NO)
si5_st5_NO = np.array(si5_st5_NO)
si7_st3_NO = np.array(si7_st3_NO)
g_array = np.array(g_array)

visible_idx = np.r_[0:10, 19, 49]

# Direct diagnilisation

In [21]:
site = 3
state = 11
g = 1

In [22]:
H_ED_s, K_ED_s, V_ED_s = hamiltonian_sparse(state, site, g)

H_ED_d = H_ED_s.toarray()
K_ED_d = K_ED_s.toarray()
V_ED_d = V_ED_s.toarray()

eig_val_ED_s, eig_vec_ED_s = np.linalg.eigh(H_ED_d) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)


In [23]:
index = np.argmin(eig_val_ED_s)
ground_energy = eig_val_ED_s[index]

ground_energy

np.float64(-2.248454586348324)

In [ ]:
# Smallest algebraic eigenpair (ground state)
eigvals, eigvecs = spla.eigsh(H_ED_s, k=1, which='SA', tol=1e-19, maxiter=2000)
E0 = eigvals[0]
psi0 = eigvecs[:, 0]

In [25]:
# Sanity check: residual
res = sp.linalg.norm(sp.csr_matrix(H_ED_s @ psi0.T - E0 * psi0) / abs(E0 if E0 != 0 else 1))
print(f"E0 = {E0:.12g}, residual = {res:.2e}")

E0 = -2.24845458635, residual = 6.65e-15


In [26]:
E0 - ground_energy

np.float64(-2.6645352591003757e-15)

In [32]:
state = 11
site = 3

# for site in range(2, 5):
#     for state in range(5, 11, 2):
for g_index in range(1, 11):
    g = g_index /10
    print(f"{site} Sites, {state} States, g = {g}:")

    H_ED_s, _, _ = hamiltonian_sparse(state, site, g)
    H_ED_d = H_ED_s.toarray()
    eig_val_ED_s, eig_vec_ED_s = np.linalg.eigh(H_ED_d)

    index = np.argmin(eig_val_ED_s)
    E0_ED = eig_val_ED_s[index]

    eig_val_DD, eig_vec_DD = spla.eigsh(H_ED_s, k=1, which='SA', tol=1e-19, maxiter=2000)
    E0_DD = eig_val_DD[0]
    psi0_DD = eig_vec_DD[:, 0]

    res = sp.linalg.norm(sp.csr_matrix(H_ED_s @ psi0_DD - E0_DD * psi0_DD) / abs(E0_DD if E0_DD != 0 else 1))
    print(f"E0 = {E0_DD:.12g}, residual = {res:.2e}")
    print("Compare ground energyes:", np.abs(E0_DD - E0_ED), "\n")


3 Sites, 11 States, g = 0.1:


/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)


E0 = -0.0201017290509, residual = 8.54e-13
Compare ground energyes: 3.0427049768633196e-15 

3 Sites, 11 States, g = 0.2:
E0 = -0.0858736144732, residual = 1.99e-13
Compare ground energyes: 9.992007221626409e-16 

3 Sites, 11 States, g = 0.3:
E0 = -0.204382901101, residual = 8.62e-14
Compare ground energyes: 8.881784197001252e-16 

3 Sites, 11 States, g = 0.4:
E0 = -0.378555048775, residual = 5.81e-14
Compare ground energyes: 6.661338147750939e-16 

3 Sites, 11 States, g = 0.5:
E0 = -0.605250827359, residual = 2.64e-14
Compare ground energyes: 3.552713678800501e-15 

3 Sites, 11 States, g = 0.6:
E0 = -0.876710970138, residual = 2.45e-14
Compare ground energyes: 4.440892098500626e-15 

3 Sites, 11 States, g = 0.7:
E0 = -1.18386696572, residual = 1.44e-14
Compare ground energyes: 6.661338147750939e-15 

3 Sites, 11 States, g = 0.8:
E0 = -1.51872111098, residual = 1.55e-14
Compare ground energyes: 1.199040866595169e-14 

3 Sites, 11 States, g = 0.9:
E0 = -1.87509239644, residual = 1.36e-1

# Natural Orbitals with direct diagnilisation test.

In [2]:
state = 3
site = 5
g = 0.6

In [7]:
H_s, K_s, V_s = hamiltonian_general_sparse(state, site, g)
H_d, K_d, V_d = hamiltonian_general_dense(state, site, g)

In [5]:
eig_val_NO_s, eig_vec_NO_s = np.linalg.eigh(H_s.toarray())
eig_val_NO_d, eig_vec_NO_d = np.linalg.eigh(H_d)

In [6]:
np.allclose(eig_val_NO_d, eig_val_NO_s, atol=1e-19)

True